In [ ]:
!pip install skorch

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Load dataset
df = pd.read_csv('./House_Rent_Dataset.csv')

In [ ]:
# Separate target and features
target = pd.DataFrame(df['Rent'])
df.drop('Rent', axis=1, inplace=True)

In [ ]:
# Encode categorical features
for col in df[['Floor', 'Area Type', 'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred', 'Point of Contact']]:
    df[col] = pd.Categorical(df[col]).codes

In [ ]:
# Convert date to numeric
df['Posted On'] = pd.to_datetime(df['Posted On'])
df['Posted On'] = df['Posted On'].astype('int64') // 10**10

In [ ]:
# Check the first few rows
print(df.head())
print(target.head())

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.values, target.values, test_size=0.2, random_state=42)

In [ ]:
# Convert Rent values to a binary class for classification (for example, high rent vs. low rent)
median_rent = target.median()[0]
y_train_class = (y_train >= median_rent).astype(int).ravel()  # Ensure 1D
y_test_class = (y_test >= median_rent).astype(int).ravel()  # Ensure 1D

In [ ]:
# Define the neural network architecture with flexible layers
class HouseRentNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(HouseRentNN, self).__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [ ]:
# Define the input and output dimensions
input_dim = X_train.shape[1]
output_dim = 2


In [ ]:
# Create the neural network
net = NeuralNetClassifier(
    module=HouseRentNN,
    max_epochs=20,
    lr=0.1,
    optimizer=optim.Adam,
    criterion=nn.CrossEntropyLoss,
    iterator_train__shuffle=True,
    verbose=0,
    module__input_dim=input_dim,
    module__output_dim=output_dim
)

In [ ]:
# Define the parameter grid for GridSearchCV
params = {
    'lr': [0.01, 0.02, 0.05],
    'max_epochs': [10, 20, 30],
    'module__hidden_dims': [
        [32],
        [64],
        [32, 32],
        [64, 32],
        [64, 64],
        [128, 64],
        [128, 128]
    ],
    'optimizer__weight_decay': [0, 1e-4, 1e-2]
}


In [ ]:
# Perform grid search
gs = GridSearchCV(net, params, refit=True, cv=3, scoring='accuracy')
gs.fit(X_train.astype(np.float32), y_train_class)

In [ ]:
# Print best parameters and best score
print("Best parameters found: ", gs.best_params_)
print("Best cross-validation accuracy: ", gs.best_score_)

In [ ]:
# Print all results
results = pd.DataFrame(gs.cv_results_)
print(results)

In [ ]:
# Evaluate the best model on the test set
best_model = gs.best_estimator_
test_accuracy = best_model.score(X_test.astype(np.float32), y_test_class)
print("Test set accuracy: ", test_accuracy)